In [11]:
#Import dependencies
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import random
from citipy import citipy
from datetime import datetime 
from config import weather_api_key
import requests


In [12]:
# Create random latitude and longitude coordinates 
latitudes = np.random.uniform(-90, 90, size = 2000)
longitudes = np.random.uniform(-180, 180, size = 2000)
coordinates = zip(latitudes, longitudes)
coordinates


In [13]:
# Add the latitudes and longitudes to a list. 
coordinates = list(coordinates)

In [14]:
# Create list for cities 
cities = []

# Use citipy to find nearest cities using coordinate list
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1])
    city_name = city.city_name
    country_name = city.country_code
    
    # Find unique cities to add to cities list
    if city_name not in cities:
        cities.append(city_name)
    
    # See cities and contries
    # print(f'{city_name}, {country_name}')

# Count amount of unique cities
len(cities)

754

In [15]:
# Create basic url for API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [18]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in our list.
for i in range(len(cities)):
    
   # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i]

   # Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log URL, set, and record numbers for each city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count += 1

# Run an API request for each of the cities.
    try:
       
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()

        # Parse out the needed data.
        city_city = city.title()
        city_country = city_weather['sys']['country']
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_hum = city_weather['main']['humidity']
        city_cloud = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_desc = city_weather['weather'][0]['description']

      # Append the city information into city_data list.
        city_data.append({"City": city_city,
                         "Country": city_country,
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_hum,
                         "Cloudiness": city_cloud,
                         "Wind Speed": city_wind,
                         "Current Description": city_desc,
                         })
        
    #If an error occurs
    except: 
        print("City not found. Skipping")
        pass

    # Complete Data Retrieval    
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

ssing Record 44 of Set 21 | nizhniy kuranakh
Processing Record 45 of Set 21 | catuday
Processing Record 46 of Set 21 | campeche
Processing Record 47 of Set 21 | kulunda
Processing Record 48 of Set 21 | ciudad bolivar
Processing Record 49 of Set 21 | zhigansk
Processing Record 50 of Set 21 | the valley
Processing Record 1 of Set 22 | araguaina
Processing Record 2 of Set 22 | kemijarvi
Processing Record 3 of Set 22 | meadow lake
Processing Record 4 of Set 22 | basco
Processing Record 5 of Set 22 | butte
Processing Record 6 of Set 22 | yuncheng
Processing Record 7 of Set 22 | prince rupert
Processing Record 8 of Set 22 | thai binh
Processing Record 9 of Set 22 | port elizabeth
Processing Record 10 of Set 22 | taree
Processing Record 11 of Set 22 | acarau
Processing Record 12 of Set 22 | northam
Processing Record 13 of Set 22 | pemba
Processing Record 14 of Set 22 | inhambane
Processing Record 15 of Set 22 | taicheng
Processing Record 16 of Set 22 | nizhneyansk
City not found. Skipping
Pro

In [21]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Upernavik,GL,72.7868,-56.1549,33.78,98,85,5.53,overcast clouds
1,Rikitea,PF,-23.1203,-134.9692,71.51,57,65,4.56,broken clouds
2,Albany,US,42.6001,-73.9662,63.18,50,42,7.00,scattered clouds
3,Asfi,MA,32.2994,-9.2372,64.67,88,24,6.93,few clouds
4,Ushuaia,AR,-54.8000,-68.3000,29.86,93,75,23.02,light snow
5,Port Alfred,ZA,-33.5906,26.8910,58.96,81,26,9.89,scattered clouds
6,Halifax,CA,44.6453,-63.5724,64.89,70,40,6.91,scattered clouds
7,Arraial Do Cabo,BR,-22.9661,-42.0278,72.27,87,1,7.81,clear sky
8,Faya,SA,18.3851,42.4509,68.11,23,7,5.10,clear sky
9,Khatanga,RU,71.9667,102.5000,29.68,85,97,10.25,overcast clouds


In [29]:
# Create CSV file of DF
csv_weather_file = "Weatherpy_database.csv"
# Export CSV file 
city_data_df.to_csv(csv_weather_file, index_label="City_ID")